In [5]:
# %matplotlib notebook
import numpy as np, imp, os, datetime as dt, pandas as pd
import matplotlib.pyplot as plt, pickle, seaborn as sb, random
from scipy.stats import norm

# Path to folder containing wrapper modules
wrapper_fpath = (r"..\inca.py")
optimize_funs_fpath = (r'..\inca_calibration.py')
emcee_Morsa_fpath = (r'simplyp_emcee_Morsa.py')

wr = imp.load_source('inca',wrapper_fpath)
cf = imp.load_source('inca_calibration', optimize_funs_fpath)
em = imp.load_source('simplyp_emcee_Morsa.py', emcee_Morsa_fpath)

In [6]:
# Setup used to run emcee

wr.initialize('simplyp.dll')

dataset = wr.DataSet.setup_from_parameter_and_input_files('../../Applications/SimplyP/Morsa/MorsaParameters.dat',
                                                          '../../Applications/SimplyP/Morsa/MorsaInputs.dat')

comparisons = [('Reach flow (daily mean, cumecs)', ['Kure'], 'Observed Q', []),
           ('Reach suspended sediment concentration', ['Kure'], 'Observed SS at Kure', []),
           ('Reach TP concentration', ['Kure'], 'Observed TP at Kure', []),
           ('Reach TDP concentration', ['Kure'], 'Observed TDP at Kure', []),
           ('Reach PP concentration', ['Kure'], 'Observed PP at Kure', []),            
          ]

# Read in csv with parameters to vary, short names and param ranges
fpath = fpath = r'C:\Data\GitHub\INCABuilder\PythonWrapper\SimplyP\SimplyP_calParams_ranges_morsa_v3.csv'
param_df = pd.read_csv(fpath)

# Make calibration variable (list of (param, index) tuples to calibrate)
calibration = [
 ('Degree-day factor for snowmelt', []),
 ('Proportion of precipitation that contributes to quick flow', []),
 ('PET multiplication factor', []),
 ('Baseflow index', []),
 ('Groundwater time constant', []),
 ('Soil water time constant', ['Semi-natural']),
 ('Reach sediment input scaling factor', []),
 ('Initial soil water TDP concentration and EPC0', ['Arable']),
 ('Groundwater TDP concentration', []),
 ('Particulate P enrichment factor', []),
 ('Reach effluent TDP inputs', ['Kure']),
 ('M_Q', []),
 ('M_SS', []),
 ('M_P', [])]

n_vars = len(comparisons)
n_Ms = 3

initial_guess = cf.default_initial_guess(dataset, calibration[:-n_Ms])
#NOTE: This reads the initial guess that was provided by the parameter file, excluding any error term parameters
# Initial guess for the residual error term(s)
for i in range(0,3):
    initial_guess.append(0.5) #3 terms, 

# Read max & min values from file
minval = list(param_df['Min'].dropna().values)
maxval = list(param_df['Max'].dropna().values)

# param_min = [0.1 * x for x in initial_guess]
# param_max = [10.0 * x for x in initial_guess]

cf.constrain_min_max(dataset, calibration, minval, maxval) #NOTE: Constrain to the min and max values recommended by the model in case we made our bounds too wide.

# Labels
labels_short = param_df['ShortName']
labels_long  = ['%s, %s' % (cal[0], cal[1]) 
                    for cal in calibration[:-n_Ms]]
labels_long.append('M_Q')
labels_long.append('M_SS')
labels_long.append('M_P')

# Objective
skiptimesteps = 30   # Skip this many first timesteps in the objective evaluation
objective = (cf.log_likelyhood, comparisons, skiptimesteps)

###########################################################
# emcee setup params
n_walk = 100
n_steps = 10000
n_burn = 5000

In [7]:
# Read in the emcee results and reshape
infile = r'pickled\v3\emcee_results_Morsa_v3.pkl'
samp, lnprob = pickle.load(open(infile,'rb')) # keys: variables, return df of timeseries, one col per station

samplelist, lnproblist = em.reshape_samples(samp, lnprob, n_burn)

In [ ]:
# Chain plot  
# em.chain_plot(samp, labels_long, "simplyp_plots\\chains.png")
# TO DO: chain plot of how log likelihood evolves for each chain (all chains on one plot)

In [ ]:
# em.triangle_plot(samplelist, labels_short, "simplyp_plots\\triangle_plot.png")

In [ ]:
def do_n_random_simulations_from_sample_list(dataset, samplelist, calibration, objective, n_samples, var_list) :

    llfun, comparisons, skiptimesteps = objective

    for i, tup in enumerate(comparisons):
        simname, simindexes, obsname, obsindexes = comparisons[i]

    n_comparisons = len(comparisons)
    
    # Set up for looping over param sets
    var_simDict_paramOnly = {} # Keys: list of variables. Must be same length as comparisons
                               # Returns: List of simulated results (each element a 1D array of model output)
    var_simDict_overall = {}
    
    for i in var_list:
        var_simDict_paramOnly[i] = []
        var_simDict_overall[i] = []
    
    # Loop over param sets in sample list
    for it in range(1, n_samples) :       #TODO: Should be paralellized, really..

        random_index = random.randint(0, len(samplelist)-1)
        random_sample = samplelist[random_index]

        cf.set_values(dataset, random_sample, calibration, n_comparisons)
        dataset.run_model()
        
        for i, tup in enumerate(comparisons):
            
            simname, simindexes, obsname, obsindexes = comparisons[i]
            sim = dataset.get_result_series(simname, simindexes)
            
            var_simDict_paramOnly[var_list[i]].append(sim)

            if i<3: # Q, SS, TP
                M = random_sample[len(calibration)-n_comparisons+i]
            else: # TDP, PP same as TP
                M = random_sample[-1]
            stoch = norm.rvs(loc=0, scale=M*sim, size=len(sim))
            perturbed = sim + stoch

            var_simDict_overall[var_list[i]].append(perturbed)

    return(var_simDict_paramOnly, var_simDict_overall)

# ###########################################################################################################
# Generate random samples
n_random_samples = 1000
varlist = ['Q','SS','TP','TDP','PP']

ar_simDict_paramOnly, var_simDict_overall = do_n_random_simulations_from_sample_list(dataset,
                                            samplelist, calibration, objective, n_random_samples, varlist)


In [ ]:
def plot_percentiles(dataset, simulationresults, calibration, objective,
                     comparison_idx, perc, filename, ylab, ymax) :

    llfun, comparisons, skiptimesteps = objective
    comparisontolookat = comparisons[comparison_idx] 
    simname, simindexes, obsname, obsindexes = comparisontolookat

    percentiles = np.percentile(simulationresults, perc, axis=0)

    fig, ax = plt.subplots(figsize=(15,6))

    start_date = dt.datetime.strptime(dataset.get_parameter_time('Start date', []),'%Y-%m-%d')
    timesteps = dataset.get_parameter_uint('Timesteps', [])
    date_idx = np.array(pd.date_range(start_date, periods=timesteps))

    ax.plot(date_idx, percentiles[1], color='red', label='median simulated', lw=1)
    ax.fill_between(date_idx, percentiles[0], percentiles[2], color='red', alpha=0.15)

    obs = dataset.get_input_series(obsname, obsindexes, True)

    # If lots of missing data in obs (i.e. not daily), plot as scatter too
    if len(obs[~np.isnan(obs)])< 0.5*len(percentiles[1]):
        marker='o'
        ax.plot(date_idx, obs, color = 'black', marker='o', ms=3, ls='None', label = 'observed')
#         ax.plot(date_idx, obs, color = 'black', lw=1)

#     else:
#         ax.plot(date_idx, obs, color = 'black', label = 'observed', lw=1)

    plt.ylim(bottom=0, top=ymax) # Stochastic error may cause negative flows; mask for now (N.B. need to revisit)

    ax.legend(fontsize=14)

    plt.setp(ax.get_xticklabels(), visible=True, rotation=30, ha='right', fontsize=12)
    plt.ylabel(ylab, fontsize=14)

    return (fig, ax)

# ####################################################################################

perc = [25, 50, 75]
var = 'PP'
fig_fpath = r'simplyp_plots/Timeseries_%s_1980-1990_withUncertainty.png' %var

var_idx_dict = {'Q':0, 'SS':1, 'TP':2, 'TDP':3,'PP':4}
ylab_dict = {'Q':'Q (m$^3$/s)', 'SS':'Suspended sediment (mg/l)', 'TP':'Total P (mg/l)',
             'TDP':'Total dissolved P (mg/l)','PP':'Particulate P (mg/l)'}
ymax_dict = {'Q':80, 'SS':300, 'TP':0.5, 'TDP':0.2,'PP':0.5}

comparison_idx = var_idx_dict[var]
ylab = ylab_dict[var]
ymax = ymax_dict[var]

fig, ax = plot_percentiles(dataset, var_simDict_overall[var], calibration, objective,
                           comparison_idx, perc,
                           "simplyp_plots\\percentiles_fullUncertainty_%s.png" %varlist[comparison_idx],
                          ylab, ymax)

plt.tight_layout()
plt.savefig(fig_fpath,dpi=300)


#     em.plot_percentiles(dataset, param_only, calibration, objective, comparison_idx, perc,
#     "simplyp_plots\\percentiles_paramUncertainty_%s.png" %varlist[comparison_idx])


In [ ]:
# varlist = ['Q','SS','TP','TDP','PP']
# for comparison_idx in range(0,n_vars):    
    #best = em.best_sample(samplelist, lnproblist)
    #em.plot_simulations(dataset, best, param_only, calibration, objective, comparison_idx, "simplyp_plots\\random_samples.png")
    #em.plot_simulations(dataset, best, overall, calibration, objective, comparison_idx, "simplyp_plots\\random_samples2.png")

In [8]:
em.GoF_stats_single_sample(samplelist, lnproblist, dataset, calibration, objective, n_vars)


Best sample (min log likelihood):

Goodness of fit for Reach flow (daily mean, cumecs) [Kure] vs Observed Q []:
Mean error (bias): 0.743289
Mean absolute error: 2.322304
Mean square error: 14.376418
Nash-Sutcliffe coefficient: 0.669567
Number of observations: 7594


Goodness of fit for Reach suspended sediment concentration [Kure] vs Observed SS at Kure []:
Mean error (bias): -21.917660
Mean absolute error: 38.686623
Mean square error: 15164.380684
Nash-Sutcliffe coefficient: 0.081440
Number of observations: 1071


Goodness of fit for Reach TP concentration [Kure] vs Observed TP at Kure []:
Mean error (bias): -0.025265
Mean absolute error: 0.055410
Mean square error: 0.014902
Nash-Sutcliffe coefficient: 0.169913
Number of observations: 1075


Goodness of fit for Reach TDP concentration [Kure] vs Observed TDP at Kure []:
Mean error (bias): 0.004143
Mean absolute error: 0.010197
Mean square error: 0.000204
Nash-Sutcliffe coefficient: -0.062376
Number of observations: 175


Goodness of f

In [ ]:
# Extract best parameter set and run model with that for varioius time periods, extracting
# GoF statistics

# Original param set: "1979-10-01" for 4017 days (11 yrs), i.e. to 30/09/1990

# Set best param set to model dataset
best_sample = em.best_sample(samplelist, lnproblist)
cf.set_values(dataset, best_sample, calibration, n_vars)

# NOTE!!!: looking at the 'best', and all the 'insensitive' param values
# are unaltered in the file compared to the manual calibration
# NEEDS DEBUGGING!!!
dataset.write_parameters_to_file('Morsa_emcee_best_parameters_v3.dat')

# NOTE: for some reason code below isn't working, get an access violation error whenever I
# try to access an input series after resetting the date params. Clunky workaround for now:
# write params to file, manually change start date and time steps, then in next cell
# read back in and do GoF...

# # Reset start date and timesteps and realign input data
# dataset.set_parameter_time('Start date', [], start_date)
# dataset.set_parameter_uint('Timesteps',[],timesteps)

# # Align inputs with new start date and time steps
# model_inputs = [i[0] for i in dataset.get_input_list()]
# for variable in model_inputs:
#     series = dataset.get_input_series(variable,[], True)
#     dataset.set_input_series(variable, [], series, True)

# # Run and extract simulated output
# dataset.run_model()

# cf.print_goodness_of_fit(dataset, objective)

In [9]:
# For comparison with INCA-P results in MARS project (they calibrated to this period)
# They started in Jan 1983; went back little bit here for hydrol year/snow
start_date = "1982-10-01"
timesteps = 11779 # 31/12/2014

dataset.delete()
dataset = wr.DataSet.setup_from_parameter_and_input_files('Morsa_emcee_best_parameters_v3.dat',
                                                          '../../Applications/SimplyP/Morsa/MorsaInputs.dat')
dataset.run_model()
cf.print_goodness_of_fit(dataset, objective)


Goodness of fit for Reach flow (daily mean, cumecs) [Kure] vs Observed Q []:
Mean error (bias): 0.832417
Mean absolute error: 2.379448
Mean square error: 15.443881
Nash-Sutcliffe coefficient: 0.685261
Number of observations: 8380


Goodness of fit for Reach suspended sediment concentration [Kure] vs Observed SS at Kure []:
Mean error (bias): -20.512779
Mean absolute error: 37.194580
Mean square error: 13337.188521
Nash-Sutcliffe coefficient: 0.089762
Number of observations: 1299


Goodness of fit for Reach TP concentration [Kure] vs Observed TP at Kure []:
Mean error (bias): -0.021700
Mean absolute error: 0.052833
Mean square error: 0.013311
Nash-Sutcliffe coefficient: 0.180552
Number of observations: 1293


Goodness of fit for Reach TDP concentration [Kure] vs Observed TDP at Kure []:
Mean error (bias): 0.004143
Mean absolute error: 0.010197
Mean square error: 0.000204
Nash-Sutcliffe coefficient: -0.062377
Number of observations: 175


Goodness of fit for Reach PP concentration [Kure

In [10]:
# Validation
start_date = "1990-10-01"
timesteps = 8765 # 30/09/2014 (24 yrs)

dataset.delete()
dataset = wr.DataSet.setup_from_parameter_and_input_files('Morsa_emcee_best_parameters_v3.dat',
                                                          '../../Applications/SimplyP/Morsa/MorsaInputs.dat')
dataset.run_model()
cf.print_goodness_of_fit(dataset, objective)


Goodness of fit for Reach flow (daily mean, cumecs) [Kure] vs Observed Q []:
Mean error (bias): 0.948659
Mean absolute error: 2.341733
Mean square error: 14.940045
Nash-Sutcliffe coefficient: 0.692034
Number of observations: 7196


Goodness of fit for Reach suspended sediment concentration [Kure] vs Observed SS at Kure []:
Mean error (bias): -12.843669
Mean absolute error: 28.084560
Mean square error: 5213.204073
Nash-Sutcliffe coefficient: 0.183914
Number of observations: 887


Goodness of fit for Reach TP concentration [Kure] vs Observed TP at Kure []:
Mean error (bias): -0.013236
Mean absolute error: 0.043389
Mean square error: 0.008343
Nash-Sutcliffe coefficient: 0.246098
Number of observations: 880


Goodness of fit for Reach TDP concentration [Kure] vs Observed TDP at Kure []:
Mean error (bias): 0.014877
Mean absolute error: 0.014877
Mean square error: 0.000247
Nash-Sutcliffe coefficient: -60.841309
Number of observations: 2


Goodness of fit for Reach PP concentration [Kure] vs

In [11]:
# Previous period I was playing with (and getting NS of around 0.7-8 for Q)
start_date = "2007-10-01"
timesteps = 730 # 30/09/2014 (24 yrs)

dataset.delete()
dataset = wr.DataSet.setup_from_parameter_and_input_files('Morsa_emcee_best_parameters_v3.dat',
                                                          '../../Applications/SimplyP/Morsa/MorsaInputs.dat')
dataset.run_model()
cf.print_goodness_of_fit(dataset, objective)


Goodness of fit for Reach flow (daily mean, cumecs) [Kure] vs Observed Q []:
Mean error (bias): 0.682677
Mean absolute error: 2.461051
Mean square error: 19.117286
Nash-Sutcliffe coefficient: 0.689973
Number of observations: 729


Goodness of fit for Reach suspended sediment concentration [Kure] vs Observed SS at Kure []:
Mean error (bias): -8.570899
Mean absolute error: 28.034603
Mean square error: 4574.054479
Nash-Sutcliffe coefficient: 0.241892
Number of observations: 93


Goodness of fit for Reach TP concentration [Kure] vs Observed TP at Kure []:
Mean error (bias): -0.010526
Mean absolute error: 0.047254
Mean square error: 0.009189
Nash-Sutcliffe coefficient: 0.350668
Number of observations: 90


Goodness of fit for Reach TDP concentration [Kure] vs Observed TDP at Kure []:
Mean error (bias): 0.014892
Mean absolute error: 0.014892
Mean square error: 0.000248
Nash-Sutcliffe coefficient: -60.988873
Number of observations: 2


Goodness of fit for Reach PP concentration [Kure] vs Obs